This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/wallaroo-testing-tutorials/abtesting).

## A/B Testing

A/B testing is a method that provides the ability to test out ML models for performance, accuracy or other useful benchmarks.  A/B testing is contrasted with the Wallaroo Shadow Deployment feature.  In both cases, two sets of models are added to a pipeline step:

* Control or Champion model:  The model currently used for inferences.
* Challenger model(s): One or more models that are to be compared to the champion model.

The two feature are different in this way:

| Feature | Description |
|---|---|
| A/B Testing | A subset of inferences are submitted to either the champion ML model or a challenger ML model, and the results of each are output as part of the `InferenceResult` `output` attribute. |
| Shadow Deploy | All inferences are submitted to the champion model and one or more challenger models.  The results for the champion model are output as part of the `InferenceResult` `output` attribute, while the results for the challenger models are output as part of the `InferenceResult` `shadow_data` attribute. |

So to repeat:  A/B testing submits *some* of the inference requests to the champion model, some to the challenger model with one set of outputs, while shadow testing submits *all* of the inference requests to champion and shadow models, and has separate outputs.

This tutorial demonstrate how to conduct A/B testing in Wallaroo.  For this example we will be using an open source model that uses an [Aloha CNN LSTM model](https://www.researchgate.net/publication/348920204_Using_Auxiliary_Inputs_in_Deep_Learning_Models_for_Detecting_DGA-based_Domain_Names) for classifying Domain names as being either legitimate or being used for nefarious purposes such as malware distribution.  

For our example, we will perform the following:

* Create a workspace for our work.
* Upload the Aloha model and a challenger model.
* Create a pipeline that can ingest our submitted data with the champion model and the challenger model set into a A/B step.
* Run a series of sample inferences to display inferences that are run through the champion model versus the challenger model, then determine which is more efficient.

All sample data and models are available through the [Wallaroo Quick Start Guide Samples repository](https://github.com/WallarooLabs/quickstartguide_samples).

## Steps

### Import libraries

Here we will import the libraries needed for this notebook.

[Arrow support](https://arrow.apache.org/) is an early preview feature of Wallaroo version 2023.1.  The variable `arrowEnabled` is used for later inference requests to submit either the Wallaroo JSON is Arrow support is disabled, or to pass either dataframe tables or Arrow data for inference requests.

Set the `os.environ["ARROW_ENABLED"]` to either `True` if Arrow support is enabled in the Wallaroo instance, or `False` if Arrow support has not been abled.  For more information on Arrow support, see the [Wallaroo Documentation site](https://docs.wallaroo.ai).

In [1]:
import wallaroo
from wallaroo.object import EntityNotFoundError
import os
import pandas as pd
# Check if arrow support is abled.

import os
os.environ["ARROW_ENABLED"]="True"
if "ARROW_ENABLED" in os.environ:
    arrowEnabled = os.environ["ARROW_ENABLED"]
else:
    arrowEnabled = False

In [13]:
wallaroo.__version__

'2023.1.1.dev0+g91707b40.d20230207'

In [2]:
import os
print(os.environ["ARROW_ENABLED"])

True


### Connect to the Wallaroo Instance

This command will be used to set up a connection to the Wallaroo cluster and allow creating and use of Wallaroo inference engines.

In [3]:
# SSO login through keycloak.  Uncomment to use.

wallarooPrefix = "sparkly-apple-3026"
wallarooSuffix = "wallaroo.community"


wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
               auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
               auth_type="sso")

# Login from within the Wallaroo JupyterHub service.  Uncomment to use.

#wl = wallaroo.Client()

### Create Workspace

We will create a workspace to manage our pipeline and models.  The following variables will set the name of our sample workspace then set it as the current workspace for all other commands.

In [4]:
prefix = "jch-"
workspace_name = f'{prefix}abtestworkspace'
pipeline_name = f'{prefix}abtestpipeline'

In [5]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

{'name': 'jch-abtestworkspace', 'id': 4, 'archived': False, 'created_by': '4a2367bf-14c3-4fe9-9494-8c44b50744b4', 'created_at': '2023-02-09T20:38:47.635715+00:00', 'models': [{'name': 'aloha-control', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2023, 2, 9, 20, 38, 52, 546061, tzinfo=tzutc()), 'created_at': datetime.datetime(2023, 2, 9, 20, 38, 52, 546061, tzinfo=tzutc())}, {'name': 'aloha-challenger', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2023, 2, 9, 20, 38, 55, 36745, tzinfo=tzutc()), 'created_at': datetime.datetime(2023, 2, 9, 20, 38, 55, 36745, tzinfo=tzutc())}], 'pipelines': [{'name': 'randomsplitpipeline-demo', 'create_time': datetime.datetime(2023, 2, 9, 20, 38, 57, 325452, tzinfo=tzutc()), 'definition': '[]'}]}

### Set Up the Champion and Challenger Models

Now we upload the Champion and Challenger models to our workspace.  We will use two models:

1. `aloha-cnn-lstm` model.
2. `aloha-cnn-lstm-new` (a retrained version)

### Set the Champion Model

We upload our champion model, labeled as `control`.

In [6]:
control =  wl.upload_model("aloha-control",   'models/aloha-cnn-lstm.zip').configure('tensorflow')

### Set the Challenger Model

Now we upload the Challenger model, labeled as `challenger`.

In [7]:
challenger = wl.upload_model("aloha-challenger",   'models/aloha-cnn-lstm-new.zip').configure('tensorflow')

### Define The Pipeline

Here we will configure a pipeline with two models and set the control model with a random split chance of receiving 2/3 of the data.  Because this is a random split, it is possible for one model or the other to receive more inferences than a strict 2:1 ratio, but the more inferences are run, the more likely it is for the proper ratio split.

In [8]:
pipeline = (wl.build_pipeline("randomsplitpipeline-demo")
            .add_random_split([(2, control), (1, challenger)], "session_id"))

### Deploy the pipeline

Now we deploy the pipeline so we can run our inference through it.

In [9]:
experiment_pipeline = pipeline.deploy()

In [10]:
experiment_pipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.24.1.137',
   'name': 'engine-69dd5cd7b7-nnrnm',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'randomsplitpipeline-demo',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'aloha-control',
      'version': '74d55753-93f3-4387-9f82-58baac290946',
      'sha': 'fd998cd5e4964bbbb4f8d29d245a8ac67df81b62be767afbceb96a03d1a01520',
      'status': 'Running'},
     {'name': 'aloha-challenger',
      'version': '827fbd98-12e5-4d1d-9e1d-4e87a82d1b37',
      'sha': '223d26869d24976942f53ccb40b432e8b7c39f9ffcf1f719f3929d7595bceaf3',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.24.1.138',
   'name': 'engine-lb-74b4969486-9bxbp',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': []}

# Run a single inference
Now we have our deployment set up let's run a single inference. In the results we will be able to see the inference results as well as which model the inference went to under model_id.  We'll run the inference request 5 times, with the odds are that the challenger model being run at least once.

In [11]:
pd.set_option('display.max_colwidth', None)

In [ ]:
#test_data = pd.read_json('data/data-1k.df.json')

#result = experiment_pipeline.infer(test_data)
#display(test_data)

In [12]:
import json
from IPython.display import display



results = []
if(arrowEnabled):
    # use dataframe JSON files
    for x in range(5):
        result = experiment_pipeline.infer_from_file("data/data-1.df.json")
        display([json.loads(result.loc[0]["out.split"][0])["name"],result.loc[0]["out.main"][0]])        
else:
    # use Wallaroo JSON files
    results.append(experiment_pipeline.infer_from_file("data/data-1.json"))
    results.append(experiment_pipeline.infer_from_file("data/data-1.json"))
    results.append(experiment_pipeline.infer_from_file("data/data-1.json"))
    results.append(experiment_pipeline.infer_from_file("data/data-1.json"))
    results.append(experiment_pipeline.infer_from_file("data/data-1.json"))
    for result in results:
        print(result[0].model())
        print(result[0].data())

HTTPError: [Errno 503 Server Error: Service Unavailable for url: https://haunted-horse-7281.api.wallaroo.dev/v1/api/pipelines/infer/randomsplitpipeline-demo-2?dataset%5B%5D=%2A&dataset.exclude%5B%5D=metadata&dataset.separator=.] 

### Run Inference Batch

We will submit 1000 rows of test data through the pipeline, then loop through the responses and display which model each inference was performed in.  The results between the control and challenger should be approximately 2:1.

In [ ]:
l = []
responses =[]
if(arrowEnabled):
    #Read in the test data as one dataframe
    test_data = pd.read_json('data/data-1k.df.json')
    # For each row, submit that row as a separate dataframe
    # Add the results to the responses array
    for index, row in test_data.iterrows():
        #display(row.to_frame('text_input').reset_index())
        responses.append(experiment_pipeline.infer(row.to_frame('text_input').reset_index()))
        #print(json.loads(result.loc[0]["out.split"][0])["name"])
        #l.append(json.loads(result.loc[0]["out.split"][0])["name"])
    #print(l)
    #now get our responses for each row
    # each r is a dataframe, then get the result from out.split into json and get the model name
    l = [json.loads(r.loc[0]["out.split"][0])["name"] for r in responses]
    df = pd.DataFrame({'model': l})
    display(df.model.value_counts())
else:
    from data import test_data
    for nth in range(1000):
        responses.extend(experiment_pipeline.infer(test_data.data[nth]))
    l = [r.raw['model_name'] for r in responses]
    df = pd.DataFrame({'model': l})
    df.model.value_counts()

### Test Challenger

Now we have run a large amount of data we can compare the results.

For this experiment we are looking for a significant change in the fraction of inferences that predicted a probability of the seventh category being high than 0.5 so we can determine whether our challenger model is more "successful" than the champion model at identifying category 7.

In [ ]:
control_count = 0
challenger_count = 0

control_success = 0
challenger_success = 0

if(arrowEnabled):
    # do nothing
    for r in responses:
        if json.loads(r.loc[0]["out.split"][0])["name"] == "aloha-control":
            control_count += 1
            if(r.loc[0]["out.main"][0] > .5):
                control_success += 1
        else:
            challenger_count += 1
            if(r.loc[0]["out.main"][0] > .5):
               challenger_success += 1
else:
    for r in responses:
        if r.raw['model_name'] == "aloha-control":
            control_count += 1
            if(r.raw['outputs'][7]['Float']['data'][0] > .5):
                control_success += 1
        else:
            challenger_count +=1
            if(r.raw['outputs'][7]['Float']['data'][0] > .5):
                challenger_success += 1
           
            
print("control class 7 prediction rate: " + str(control_success/control_count))
print("challenger class 7 prediction rate: " + str(challenger_success/challenger_count))

### Undeploy Pipeline

With the testing complete, we undeploy the pipeline to return the resources back to the environment.

In [ ]:
experiment_pipeline.undeploy()
